# Imputation

In [ ]:
# loading libraries
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error

# file paths (Colab environment)
DATA_PATH = "/content/PDA_Data.xlsx"
OUT_PATH = "/content/PDA_Data_imputed_median.xlsx"

# main target and features used for grouping
TARGET_COL = "favourite_dish_price"
FEATURE_COLS = ["city", "Services", "Restaurantkonzept", "Küchenregion", "Kettenzuordnung"]

# reproducibility + CV setup
RANDOM_SEED = 42
N_SPLITS = 5

# thresholds for grouping stability
MIN_CITY_COUNT = 20          # minimum number of observations per city before keeping its own label
MIN_TRIPLE_COUNT = 10        # minimum size for (city, region, concept) group
CITY_SMOOTH_K = 20.0         # smoothing factor for city-level fallback


def normalize_text(x):
    # basic cleanup for categorical fields (remove extra spaces etc.)
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    return s


def to_numeric_price(series: pd.Series) -> pd.Series:
    # convert price strings like "12,50 €" or "12.50 EUR" into float
    def _parse(v):
        if pd.isna(v):
            return np.nan
        t = str(v).strip()
        if t == "":
            return np.nan

        # remove currency symbols and unify decimal separator
        t = t.replace("€", "").replace("EUR", "").strip()
        t = t.replace(",", ".")
        t = re.sub(r"[^0-9.\-]", "", t)

        try:
            return float(t)
        except Exception:
            # fallback in case parsing fails
            return np.nan

    return series.map(_parse).astype(float)


def reduce_rare_levels(series: pd.Series, min_count: int, other_label: str = "Other") -> pd.Series:
    # group very small categories into "Other" to avoid unstable medians
    vc = series.value_counts(dropna=True)
    keep = set(vc[vc >= min_count].index.astype(str).tolist())

    def _map(v):
        if pd.isna(v):
            return np.nan
        sv = str(v)
        return sv if sv in keep else other_label

    return series.map(_map)


def make_strat_bins(y: np.ndarray, n_bins: int = 10) -> np.ndarray:
    # create quantile-based bins for stratified CV on continuous target
    y = np.asarray(y, dtype=float)
    ok = y[~np.isnan(y)]
    if ok.size == 0:
        return np.zeros_like(y, dtype=int)

    q = np.linspace(0, 1, n_bins + 1)
    edges = np.quantile(ok, q)
    edges = np.unique(edges)

    if len(edges) <= 2:
        # fallback if not enough variation
        return np.zeros_like(y, dtype=int)

    return np.digitize(y, edges[1:-1], right=True)


def eval_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
    # simple regression metrics for evaluation
    mae = float(mean_absolute_error(y_true, y_pred))
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    return {"MAE": mae, "RMSE": rmse}


def fit_imputer(train_df: pd.DataFrame, target_col: str):
    # compute global median as ultimate fallback
    global_med = float(np.nanmedian(train_df[target_col].values))

    grp2_cols = ["Küchenregion", "Restaurantkonzept"]
    grp3_cols = ["city", "Küchenregion", "Restaurantkonzept"]

    # hierarchical group statistics
    g2 = train_df.groupby(grp2_cols, dropna=False)[target_col].agg(["median", "count"])
    g3 = train_df.groupby(grp3_cols, dropna=False)[target_col].agg(["median", "count"])
    city_stats = train_df.groupby("city", dropna=False)[target_col].agg(["median", "count"])

    def city_smoothed(city):
        # smoothed city-level estimate (Bayesian-style shrinkage to global median)
        if city in city_stats.index:
            m = float(city_stats.loc[city, "median"])
            c = float(city_stats.loc[city, "count"])
            if np.isnan(m):
                return global_med
            return (m * c + global_med * CITY_SMOOTH_K) / (c + CITY_SMOOTH_K)
        return global_med

    def predict_row(row):
        # first try full triple (city, region, concept)
        k3 = (row["city"], row["Küchenregion"], row["Restaurantkonzept"])
        if k3 in g3.index:
            med3 = g3.loc[k3, "median"]
            cnt3 = g3.loc[k3, "count"]
            if pd.notna(med3) and float(cnt3) >= MIN_TRIPLE_COUNT:
                return float(med3)

        # then fallback to (region, concept)
        k2 = (row["Küchenregion"], row["Restaurantkonzept"])
        if k2 in g2.index:
            med2 = g2.loc[k2, "median"]
            if pd.notna(med2):
                return float(med2)

        # final fallback: smoothed city estimate
        return float(city_smoothed(row["city"]))

    return global_med, predict_row


def impute_prices(train_df: pd.DataFrame, df_to_impute: pd.DataFrame, target_col: str) -> np.ndarray:
    # fit hierarchical medians on train and apply to new data
    _, pred_fn = fit_imputer(train_df, target_col)
    return df_to_impute.apply(pred_fn, axis=1).astype(float).values


def main():
    # read Excel file
    df = pd.read_excel(DATA_PATH)

    # normalize categorical text columns
    for c in FEATURE_COLS:
        df[c] = df[c].map(normalize_text)

    # parse price column
    df[TARGET_COL] = to_numeric_price(df[TARGET_COL])

    # reduce rare cities
    df["city"] = reduce_rare_levels(df["city"], min_count=MIN_CITY_COUNT, other_label="Other")

    # fill remaining missing categorical values
    for c in FEATURE_COLS:
        df[c] = df[c].fillna("Missing").astype(str)

    # split into labeled and unlabeled parts
    missing_mask = df[TARGET_COL].isna()
    labeled = df[~missing_mask].copy()
    unlabeled = df[missing_mask].copy()

    # prepare stratified CV on continuous target
    y = labeled[TARGET_COL].values.astype(float)
    bins = make_strat_bins(y, n_bins=10)

    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
    oof = np.zeros_like(y, dtype=float)

    # cross-validation loop
    for tr_idx, va_idx in skf.split(labeled, bins):
        tr = labeled.iloc[tr_idx].copy()
        va = labeled.iloc[va_idx].copy()
        oof[va_idx] = impute_prices(tr, va, TARGET_COL)

    # evaluate CV performance
    metrics = eval_metrics(y, oof)
    print(f"CV (Hierarchical Median Imputation) | MAE={metrics['MAE']:.4f} | RMSE={metrics['RMSE']:.4f}")

    # final imputation for missing values using full labeled data
    pred_missing = impute_prices(labeled, unlabeled, TARGET_COL)
    pred_missing = np.clip(pred_missing, 0, None)

    df_out = df.copy()

    # overwrite the target column with imputed values where missing
    df_out.loc[missing_mask, TARGET_COL] = pred_missing

    # track origin of price value (original vs imputed)
    df_out["Preisquelle"] = np.where(missing_mask, "imputiert", "original")

    # derive 3-tier price category (tertiles)
    labels_de = ["niedrig", "mittel", "hoch"]
    df_out["Preiskategorie"] = pd.qcut(df_out[TARGET_COL].astype(float), q=3, labels=labels_de)

    # save final dataset
    df_out.to_excel(OUT_PATH, index=False)
    print(f"Saved: {OUT_PATH}")


if __name__ == "__main__":
    main()